In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
(ds_test, ds_val, ds_train), ds_info = tfds.load(
    'mnist',
    split=['train[:10%]','train[10%:25%]','train[25%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
print("train set = ", len(ds_train))
print("test set = ", len(ds_test))
print("validation set = ", len(ds_val))

train set =  45000
test set =  6000
validation set =  9000


In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

print(ds_train)

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


In [4]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [5]:
ds_val = ds_val.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_val = ds_val.batch(128)
ds_val = ds_val.cache()
ds_val = ds_val.prefetch(tf.data.AUTOTUNE)

In [6]:
print(ds_train)
print(ds_test)
print(ds_val)

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


In [7]:
shape = np.array([28,28,1])
model = tf.keras.models.Sequential()
def build(shape):
        model.add(tf.keras.layers.Conv2D(6,4,activation = 'relu',input_shape=(shape[0],shape[1],shape[2])))
        #model.add(tf.keras.layers.Conv2D(6,4,activation = 'relu'))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128,activation='relu'))
        #model.add(tf.keras.layers.Dense(64))
        model.add(tf.keras.layers.Dense(10))    
        return model

build(shape)

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_val,
)

Epoch 1/10
352/352 [==============================] - 9s 16ms/step - loss: 0.2753 - sparse_categorical_accuracy: 0.9194 - val_loss: 0.1353 - val_sparse_categorical_accuracy: 0.9586
Epoch 2/10
352/352 [==============================] - 5s 14ms/step - loss: 0.0906 - sparse_categorical_accuracy: 0.9729 - val_loss: 0.0868 - val_sparse_categorical_accuracy: 0.9738
Epoch 3/10
352/352 [==============================] - 5s 15ms/step - loss: 0.0599 - sparse_categorical_accuracy: 0.9818 - val_loss: 0.0775 - val_sparse_categorical_accuracy: 0.9776
Epoch 4/10
352/352 [==============================] - 5s 14ms/step - loss: 0.0425 - sparse_categorical_accuracy: 0.9877 - val_loss: 0.0631 - val_sparse_categorical_accuracy: 0.9814
Epoch 5/10
352/352 [==============================] - 5s 15ms/step - loss: 0.0311 - sparse_categorical_accuracy: 0.9906 - val_loss: 0.0756 - val_sparse_categorical_accuracy: 0.9784
Epoch 6/10
352/352 [==============================] - 6s 17ms/step - loss: 0.0244 - sparse_cate

In [10]:
model.save('./modelmnist')

INFO:tensorflow:Assets written to: ./modelmnist\assets


INFO:tensorflow:Assets written to: ./modelmnist\assets


In [11]:
model.evaluate(ds_test)

47/47 [==============================] - 1s 11ms/step - loss: 0.0777 - sparse_categorical_accuracy: 0.9830


[0.07772678881883621, 0.9829999804496765]

In [48]:
modeltest = tf.keras.models.load_model('./modelmnist')

In [49]:
modeltest.evaluate(ds_test)

47/47 [==============================] - 1s 9ms/step - loss: 0.0763 - sparse_categorical_accuracy: 0.9788


[0.07632710784673691, 0.9788333177566528]